# AutoEncoder for Scientific Data Reduction
-----------------------------------------------------------------

## Second Prototye - Mark II

### Updated on June 15th

- Refer to TF_code to revise the model;
- Add default graph to make sure restored model can have all the trained variables;
- Data handling: txt --> minibatch
- Make sure global step is correct
- Loss and Reconstruction Visualization


In [1]:
from __future__ import division, print_function, absolute_import
import scipy.stats as st
import math
import itertools
# import adios as ad # not installed on mac
import numpy as np
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import time
import os

# Training Parameters
learning_rate = 0.01
# num_steps = 10
num_steps = 900
# batch_size = 10

# display_step = 1000
display_step = 10
# examples_to_show = 10

# Network Parameters
num_hidden_1 = 512 # 1st layer num features
num_hidden_2 = 256 # 2nd layer num features (the latent dim)

# gromacs
# num_input = 33876*3
# num_input = 33876 
data_path = "./data"

# mnist 
num_input = 1000

global_step = tf.train.get_or_create_global_step()

In [2]:
# Gromacs data
def minibatch(fname,data_path,d):
    # set d as number of input for testing.
    # output batch will have size of [10*f*1]
    with open(os.path.join(data_path,fname)) as f:
        data = f.readlines()

    data = [line.split(" ")[1:4] for line in data]
    data = [[float(num) for num in line] for line in data]
    #print(data[0])
    #print(np.shape(data))

    # for the 3rd dimension, : for 3d, 0/1/2 for 1d
    batch_0 = np.reshape(data,[300,33876,3],order='c')[:,0:d,0]
    batch_1 = np.reshape(data,[300,33876,3],order='c')[:,0:d,1]
    batch_2 = np.reshape(data,[300,33876,3],order='c')[:,0:d,2]
    batch = np.concatenate((batch_0,batch_1,batch_2))
    print(np.shape(batch))
    return batch

In [3]:
train_batch = minibatch('md_0_2_new_0.txt',data_path,num_input)
np.shape(train_batch)

FileNotFoundError: [Errno 2] No such file or directory: './data/md_0_2_new_0.txt'

In [ ]:
# g = tf.Graph()
# with g.as_default():
X = tf.placeholder("float", [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input]))
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input]))
}

# Building the encoder
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2

# Building the decoder
def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
y_true = X

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2)) # relative loss
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)
l_sum = tf.summary.scalar('loss',loss)
w1 = tf.summary.tensor_summary('weights_eh1',weights['encoder_h1'])
w2 = tf.summary.tensor_summary('weights_eh2',weights['encoder_h2'])
w3 = tf.summary.tensor_summary('weights_dh1',weights['decoder_h1'])
w4 = tf.summary.tensor_summary('weights_dh2',weights['decoder_h2'])

# Initialize the variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# Start Training
# Start a new TF session
with tf.Session() as sess:
    sess.run(init)
    merged = tf.summary.merge([l_sum,w1,w2,w3,w4])
    writer = tf.summary.FileWriter('./graph', sess.graph)
    # Run the initializer
    saver.save(sess,'./my-model/mark-2',global_step = global_step)
    # Training
    train_batch = minibatch('md_0_2_new_0.txt',data_path,num_input)
    for i in range(0, num_steps):
#         fname = "md_0_2_new."+str(i)
        
#         train_batch =  minibatch(fname,data_path,num_input)
#         for j in range(0,10):
        summary, _, l = sess.run([merged, optimizer, loss], feed_dict={X:np.reshape(train_batch[i],[-1,num_input])})
        writer.add_summary(summary,i)
        print('Step %i: Minibatch Loss: %f' % (i, l))

    # Testing
#     for i in range(25,30):
#         fname = "md_0_2_new."+str(i)
#         test_batch =  minibatch(fname,data_path,num_input)
#         for j in range(0,10):
#             los = sess.run(loss, feed_dict={X:np.reshape(train_batch[j],[-1,num_input])})
#             print('Step %i: Validation Loss: %f' % (i, los))

In [1]:
import os
import tensorflow as tf
import numpy as np

# data_dir = "../data/GROMACS_data/"

# filenames = [os.path.join(data_dir,'md_0_2_new_%d.txt' % i) for i in range(0,4)]
# # filenames = [os.path.join(data_dir,'test.txt')]
             
# filename_queue = tf.train.string_input_producer(filenames)
# reader = tf.TextLineReader()

# values = []
# for i in range(0,300):
#     values.append(reader.read(filename_queue)[1])

# data = tf.decode_csv(values,[[1],[1.0],[1.0],[1.0]],field_delim=' ')

In [36]:
data

[<tf.Tensor 'DecodeCSV_8:0' shape=(300,) dtype=int32>,
 <tf.Tensor 'DecodeCSV_8:1' shape=(300,) dtype=float32>,
 <tf.Tensor 'DecodeCSV_8:2' shape=(300,) dtype=float32>,
 <tf.Tensor 'DecodeCSV_8:3' shape=(300,) dtype=float32>]

In [40]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    val = sess.run(data[1:4])
    print(val)
    coord.request_stop()
    coord.join(threads)

[array([4.11252, 4.1275 , 4.13719, 4.16056, 4.10783, 4.2665 , 4.35712,
       3.92583, 3.59225, 4.29996, 4.30642, 4.42061, 4.20973, 4.49416,
       4.49173, 4.38675, 4.5556 , 4.50025, 4.216  , 4.29549, 4.08479,
       4.01799, 4.0246 , 4.07355, 3.85189, 3.8746 , 3.8236 , 3.62844,
       3.87002, 4.22495, 4.4469 , 4.32676, 4.38594, 4.52268, 4.26975,
       4.41756, 4.39938, 4.1815 , 4.14966, 3.95461, 3.9137 , 3.96865,
       3.6962 , 3.74129, 4.20561, 4.29785, 4.93228, 4.14651, 4.22313,
       4.2571 , 4.22234, 4.36006, 4.34128, 4.38334, 4.30257, 4.50379,
       4.52719, 3.93405, 4.00956, 4.06294, 3.86951, 3.72921, 3.82505,
       3.63451, 3.80685, 3.7324 , 3.92416, 3.8114 , 4.09515, 3.93616,
       3.88783, 4.40266, 4.25137, 4.04623, 4.03714, 4.43227, 4.09259,
       4.22901, 4.52542, 4.44348, 4.9611 , 4.6216 , 4.36859, 4.4877 ,
       4.25189, 4.30835, 4.3344 , 4.25574, 4.29178, 4.2333 , 4.32352,
       4.33763, 4.34288, 4.25562, 4.19093, 4.25416, 4.22423, 4.2297 ,
       4.03944, 3.9

In [38]:
data

[<tf.Tensor 'DecodeCSV_8:0' shape=(300,) dtype=int32>,
 <tf.Tensor 'DecodeCSV_8:1' shape=(300,) dtype=float32>,
 <tf.Tensor 'DecodeCSV_8:2' shape=(300,) dtype=float32>,
 <tf.Tensor 'DecodeCSV_8:3' shape=(300,) dtype=float32>]

In [39]:
data[1:4]

[<tf.Tensor 'DecodeCSV_8:1' shape=(300,) dtype=float32>,
 <tf.Tensor 'DecodeCSV_8:2' shape=(300,) dtype=float32>,
 <tf.Tensor 'DecodeCSV_8:3' shape=(300,) dtype=float32>]

In [44]:
t1 = [[1, 2, 3]]
t2 = [[7, 8, 9]]

tf.concat([t1, t2], 0)


<tf.Tensor 'concat_3:0' shape=(2, 3) dtype=int32>

In [45]:
tf.concat([t1, t2], 1)

<tf.Tensor 'concat_4:0' shape=(1, 6) dtype=int32>

In [49]:
t = [[1,3,4],[3,4,5],[5,6,7]]

tf.concat(t,0)

<tf.Tensor 'concat_8:0' shape=(9,) dtype=int32>

In [55]:
data[1:4]

[<tf.Tensor 'DecodeCSV_8:1' shape=(300,) dtype=float32>,
 <tf.Tensor 'DecodeCSV_8:2' shape=(300,) dtype=float32>,
 <tf.Tensor 'DecodeCSV_8:3' shape=(300,) dtype=float32>]

In [58]:
dat = tf.concat(data[1:4],0)

In [59]:
print(dat)

Tensor("concat_15:0", shape=(900,), dtype=float32)


In [60]:
da = tf.reshape(dat,[300,3])

In [61]:
da

<tf.Tensor 'Reshape:0' shape=(300, 3) dtype=float32>

In [ ]:
tf.split(0,10,10)

In [3]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])
print(t)

Tensor("Const:0", shape=(3, 2, 3), dtype=int32)


In [19]:
t1 = tf.slice(t,[2,1,0],[1,1,3])

In [20]:
t1

<tf.Tensor 'Slice_4:0' shape=(1, 1, 3) dtype=int32>

In [21]:
with tf.Session() as sess:
    t0 = sess.run(t1)

In [22]:
t0

array([[[6, 6, 6]]], dtype=int32)